# Mastering GDAL Tools

This notebook contains the command-line workflows covered in the [Mastering GDAL Tools](https://courses.spatialthoughts.com/gdal-tools.html#) course. The notebook environment provides a great interactive learning environment for reproducing these workflows and maybe helpful to many people. You may use this as an alternative to the Terminal-based workflow taught in the course.

## Running Commands in a Notebook

You can run the GDAL commands in a Jupyter/Notebook environment by using the *Shell Assignment Syntax* - prefixing the commands with `!`. Any code cell starting with `!` will be run as a command in the shell.

For example, to run the following command `gdalinfo --version`, run it as follows in a notebook cell.

```
!gdalinfo --version
```

Jupyter Notebooks allows users to run [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) that are useful when using command-line workflows.

For changing directories, you can use the Jupyter line magic `%cd`.

```
%cd gdal-tools
```

Multiline shell commands can be run using the `%%bash` cell magic.

```
%%bash
cd gdal-tools
ls
```

## Setup and Data Download

Install GDAL Binaries

In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
  !apt-get install gdal-bin
  !pip install leafmap[raster]

Verify that GDAL Tools are installed correctly.

In [ ]:
!gdalinfo --version

Download and Unzip the Data Package

In [ ]:
import os
import zipfile
import leafmap
import matplotlib.pyplot as plt
import rasterio

In [ ]:
def download(url):
    filename = os.path.join(os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)


data_pkg_zip = 'gdal-tools.zip'
data_url = 'https://storage.googleapis.com/spatialthoughts-public-data/gdal/'


download(data_url + data_pkg_zip)

In [ ]:
with zipfile.ZipFile(data_pkg_zip) as zf:
  foldername = [info.filename for info in zf.infolist() if info.is_dir()][0]
  # Extract all the data
  zf.extractall()

print(f'Extracted the files to {foldername}.')

Once extracted, navigate to the data package folder. Here we set the `DATA_FOLDER` varaible to the full path in Google Colab. If you are using the notebook on your local machine, replace the path with the location of the unzipped folder.


In [ ]:
DATA_FOLDER = '/content/gdal-tools'
%cd $DATA_FOLDER

## 1. GDAL Tools



### 1.1 Basic Raster Processing

In [ ]:
%cd $DATA_FOLDER/srtm

In [ ]:
!gdalinfo N28E086.hgt

In [ ]:
!gdalinfo -stats N28E086.hgt

### 1.1.1 Merging Tiles

In [ ]:
!ls *.hgt > filelist.txt

In [ ]:
!cat filelist.txt

In [ ]:
!gdalbuildvrt -input_file_list filelist.txt merged.vrt

### 1.1.2 Converting Formats

In [ ]:
!gdal_translate -of GTiff merged.vrt merged.tif

In [ ]:
!ls -lh merged.tif

### 1.1.3 Compressing Output

In [ ]:
!gdal_translate -of GTiff merged.vrt merged.tif -co COMPRESS=DEFLATE

In [ ]:
!ls -lh merged.tif

In [ ]:
!gdal_translate -of GTiff merged.vrt merged.tif \
  -co COMPRESS=DEFLATE -co TILED=YES -co PREDICTOR=2

In [ ]:
!ls -lh merged.tif

### 1.1.4 Setting NoData Values

In [ ]:
!gdal_translate -of GTiff merged.vrt merged.tif \
  -co COMPRESS=DEFLATE -co TILED=YES -co PREDICTOR=2 -a_nodata -9999

In [ ]:
!gdalinfo merged.tif

### 1.1.5 Writing Cloud-Optimized GeoTIFF (COG)


In [ ]:
!gdal_translate -of COG merged.vrt merged_cog.tif \
  -co COMPRESS=DEFLATE -co PREDICTOR=2 -co NUM_THREADS=ALL_CPUS \
  -a_nodata -9999

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('merged_cog.tif', colormap='gray', layer_name='DEM')
m

### 1.2.1 Creating Hillshade


In [ ]:
!gdaldem hillshade -of COG merged.tif hillshade.tif -s 111120

In [ ]:
!gdaldem hillshade -of COG merged.tif hillshade_combined.tif -s 111120 -multidirectional

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('hillshade.tif', colormap='gray', layer_name='Hillshade')
m.add_raster('hillshade_combined.tif', colormap='gray', layer_name='Multidirectional Hillshade')
m

### 1.2.2 Creating Color Relief


In [ ]:
%%bash
cat << EOF > colormap.txt
1000,101,146,82
1500,190,202,130
2000,241,225,145
2500,244,200,126
3000,197,147,117
4000,204,169,170
5000,251,238,253
6000,255,255,255
EOF

In [ ]:
!gdaldem color-relief -of COG  merged.tif colormap.txt colorized.tif

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('colorized.tif', layer_name='Color Relief')
m

## 1.3 Processing Aerial Imagery


In [ ]:
%cd $DATA_FOLDER/naip

In [ ]:
!ls *.jp2 > filelist.txt

In [ ]:
!gdalbuildvrt -addalpha -input_file_list filelist.txt naip.vrt

In [ ]:
!gdal_translate -b 1 -b 2 -b 3 -of JPEG -outsize 2% 2% naip.vrt naip_preview.jpg

In [ ]:
from PIL import Image
from IPython.display import display
img = Image.open('naip_preview.jpg')
display(img)

### 1.3.2 Create a Tile Index


In [ ]:
!gdaltindex -write_absolute_path index.shp --optfile filelist.txt

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_vector('index.shp', layer_name='Tile Index')
m

In [ ]:
!gdalwarp -cutline aoi.shp  -crop_to_cutline naip.vrt aoi_cut.tif -co COMPRESS=DEFLATE -co TILED=YES

In [ ]:
!gdal_translate aoi_cut.tif aoi.tif \
  -co COMPRESS=JPEG -co TILED=YES -co PHOTOMETRIC=YCBCR -co JPEG_QUALITY=75 \
  -b 1 -b 2 -b 3 -mask 4 --config GDAL_TIFF_INTERNAL_MASK YES

### 1.3.5 Creating Overviews

In [ ]:
!gdaladdo aoi.tif

In [ ]:
!gdaladdo -r bilinear --config COMPRESS_OVERVIEW JPEG aoi.tif

In [ ]:
!gdal_translate -of COG aoi.tif aoi_cog.tif -co NUM_THREADS=ALL_CPUS

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('aoi_cog.tif', layer_name='Aerial Mosaic')
m

## 1.4 Processing Satellite Imagery


In [ ]:
%cd $DATA_FOLDER/landsat8

### 1.4.1 Merging individual bands into RGB composite


In [ ]:
!gdalbuildvrt -o rgb.vrt -separate \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B3.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B2.tif

In [ ]:
!gdal_translate -of COG rgb.vrt rgb.tif -co COMPRESS=DEFLATE

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('rgb.tif', layer_name='RGB Composite')
m

### 1.4.2 Apply Histogram Stretch and Color Correction

In [ ]:
!gdal_translate -scale 0 0.3 0 255 -ot Byte rgb.tif rgb_stretch.tif

In [ ]:
!gdal_translate -scale 0 0.3 0 255 -exponent 0.5 -ot Byte rgb.tif rgb_stretch.tif

In [ ]:
!gdal_translate -of COG rgb_stretch.tif rgb_stretch_cog.tif -co NUM_THREADS=ALL_CPUS

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('rgb_stretch_cog.tif', layer_name='RGB Stretched')
m

### 1.4.3 Raster Algebra

In [ ]:
!ls

In [ ]:
!gdalinfo -stats RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif

In [ ]:
!gdal_calc.py -A RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B5.tif \
  -B RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif \
  --outfile ndvi.tif --calc="(A-B)/(A+B)" --NoDataValue=-999

In [ ]:
!gdal_translate -of COG ndvi.tif ndvi_cog.tif -co NUM_THREADS=ALL_CPUS

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('ndvi_cog.tif', colormap='Greens', vmin=0, vmax=0.8, layer_name='NDVI')
m

### 1.4.4 Pan Sharpening


In [ ]:
!gdal_pansharpen.py RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B8.tif \
  rgb.tif pansharpened.tif -r bilinear -co COMPRESS=DEFLATE -co PHOTOMETRIC=RGB

In [ ]:
!gdal_translate -scale 0 0.3 0 255 -exponent 0.5 -ot Byte -a_nodata 0 \
  pansharpened.tif pansharpened_stretch.tif

In [ ]:
!gdal_translate -of COG pansharpened_stretch.tif pansharpened_stretch_cog.tif -co NUM_THREADS=ALL_CPUS

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('rgb_stretch_cog.tif', layer_name='Original')
m.add_raster('pansharpened_stretch_cog.tif', layer_name='Pansharpened')
m

## 1.5 Processing WMS Layers

In [ ]:
%cd $DATA_FOLDER

### 1.5.1 Listing WMS Layers


In [ ]:
!gdalinfo "WMS:https://sedac.ciesin.columbia.edu/geoserver/wms?version=1.3.0"

In [ ]:
!gdalinfo "WMS:https://sedac.ciesin.columbia.edu/geoserver/wms?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=grand-v1%3Agrand-v1-reservoirs-rev01&CRS=CRS:84&BBOX=-153.037,-45.881,176.825,70.398"

### 1.5.2 Creating a Service Description File

In [ ]:
!gdal_translate -of WMS "WMS:https://sedac.ciesin.columbia.edu/geoserver/wms?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=grand-v1%3Agrand-v1-reservoirs-rev01&CRS=CRS:84&BBOX=-153.037,-45.881,176.825,70.398" reservoirs.xml

### 1.5.3 Downloading WMS Layers


In [ ]:
!gdalwarp -tr 0.1 0.1 reservoirs.xml reservoirs.tif -co COMPRESS=DEFLATE -co TILED=YES

In [ ]:
!gdalwarp -tr 0.005 0.005 -te 68.106 6.762 97.412 37.078 reservoirs.xml reservoirs_india.tif -co COMPRESS=DEFLATE -co TILED=YES

In [ ]:
!gdal_translate -of COG reservoirs_india.tif reservoirs_india_cog.tif -co NUM_THREADS=ALL_CPUS

In [ ]:
m = leafmap.Map(width=800, height=500)
m.add_raster('reservoirs_india_cog.tif', layer_name='Downloaded WMS Layer')
m

## 1.6 Georeferencing


### 1.6.1 Georeferencing Images with Bounding Box Coordinates


In [ ]:
!gdalinfo earth_at_night.jpg

In [ ]:
!gdal_translate -a_ullr -180 90 180 -90 -a_srs EPSG:4326 \
  earth_at_night.jpg earth_at_night.tif \
  -co COMPRESS=JPEG -co TILED=YES -co PHOTOMETRIC=RGB

In [ ]:
!gdalinfo earth_at_night.tif

### 1.6.2 Georeferencing with GCPs


In [ ]:
!gdal_translate -gcp 418 893 70 15 -gcp 380 2432 70 5 -gcp 3453 2434  90 5 \
  -gcp 3407 895 90 15 -gcp 2662 911 85 15 \
  1870_southern-india.jpg india-with-gcp.tif

In [ ]:
!gdalwarp -t_srs EPSG:4042 -order 1 -tr 0.005 0.005 \
  india-with-gcp.tif india-reprojected-polynomial.tif \
  -co COMPRESS=JPEG -co JPEG_QUALITY=50 -co PHOTOMETRIC=YCBCR